In [1]:
# 1200x2200

DATA_INICIO = '2018-01-01 00:00:00'
DATA_FIM = '2025-12-31 23:59:59'
PALAVRAS_CHAVE = ["Vlr", "Valor", "vlr", "valor"]

In [2]:
# 1200x2200

import pandas as pd

a = pd.read_parquet('C:\\AFL_GABRIEL\\csv_parser\\DATA\\sis\\s1200')
b = pd.read_parquet('C:\\AFL_GABRIEL\\csv_parser\\DATA\\sis\\s2200')

print(len(a))
print(len(b))

264332
1058


In [3]:
# 1200x2200

import pandas as pd
import pyperclip
import os

relatorios = ['1200', '2200']

sys_df_2200 = None
sys_df_1200 = None
df_sys = None
parquet_path_sys = None

for relatorio in relatorios:
    relatorio_sis = f's{relatorio}'

    try:
        parquet_path_sys = f'C:\\AFL_GABRIEL\\csv_parser\\DATA\\sis\\{relatorio_sis}\\'

        df_sys = pd.read_parquet(parquet_path_sys, engine='pyarrow')

        df_sys['periodo'] = (
                df_sys['id'].astype(str).str[17:21] + '-' +
                df_sys['id'].astype(str).str[21:23] + '-' +
                df_sys['id'].astype(str).str[23:25] + ' ' +
                df_sys['id'].astype(str).str[25:27] + ':00'
        )

        df_sys['periodo'] = pd.to_datetime(df_sys['periodo'])
        dt_ini = pd.to_datetime(DATA_INICIO)
        dt_fim = pd.to_datetime(DATA_FIM)

        if relatorio != '2200':
            df_sys = df_sys[(df_sys['periodo'] >= dt_ini) & (df_sys['periodo'] <= dt_fim)]

    except Exception as e:
        print(f"Erro ao carregar {parquet_path_sys}: {e}")

    if relatorio == '2200':
        sys_df_2200 = df_sys
    elif relatorio == '1200':
        sys_df_1200 = df_sys

# Renomeia colunas
s1200 = sys_df_1200.copy().reset_index()
s2200 = sys_df_2200.copy()


In [4]:
# 1200x2200

s1200.rename(columns={
            'periodo': 'periodo_1200',
            'id': 'id_1200'
        }, inplace=True)

s2200.rename(columns={
            'periodo': 'periodo_2200',
            'id': 'id_2200'
        }, inplace=True)

s1200['periodo_1200'] = pd.to_datetime(s1200['periodo_1200'])
s2200['periodo_2200'] = pd.to_datetime(s2200['periodo_2200'])

# 1ª Consolidação: período_1010 <= período
merged = s1200.merge(
            s2200,
            on=['cpf_trabalhador'],
            suffixes=('_1200', '_2200')
        )

merged.drop_duplicates(subset=['index', 'cpf_trabalhador'], inplace=True)

s12_final = merged

colunas_s1200 = [col for col in sys_df_1200.columns]

s12_final.rename(columns={
            'id_1200': 'id',
            'numero_do_recibo_retificado_1200': 'numero_do_recibo_retificado',
            'numero_do_recibo_2200': 'numero_do_recibo',
            'periodo_1200': 'periodo'
        }, inplace=True)

# Juntar com as 3 colunas desejadas do s1010
colunas_final = colunas_s1200 + [
            'index', 'nome_cargo', 'cbo_cargo', 'codigo_cargo'
        ]

s12_final = s12_final[colunas_final]

# 2ª Consolidação: restante do S1200

s1200_2_ = s1200[~s1200['index'].isin(s12_final['index'])]

merged = s1200_2_.merge(
            s2200,
            on=['cpf_trabalhador'],
            suffixes=('_1200', '_1010'),
            how='left'
        )

merged.rename(columns={
            'periodo_1200': 'periodo',
            'id_1200': 'id',
            'numero_do_recibo_retificado_1200': 'numero_do_recibo_retificado',
            'numero_do_recibo_1200': 'numero_do_recibo'
        }, inplace=True)

colunas_final = [col for col in colunas_final if col not in ['periodo', 'index']]

merged_selected = merged[colunas_final]

merged_com_s12_final = pd.concat([s12_final, merged_selected])

df_final_ = merged_com_s12_final.drop(columns=['index', 'periodo'], errors='ignore').fillna('')


In [5]:
# 1200x2200

num_linhas, num_colunas = df_final_.shape
resultado = f"🔹 Total de Linhas: {num_linhas}\n🔹 Total de Colunas: {num_colunas}\n\n📊 Detalhes das Colunas:\n"

for coluna in df_final_.columns:
    linhas_preenchidas = df_final_[coluna].count()  # Conta valores não nulos
    
    # Verifica se a coluna está na lista de palavras-chave
    if any(palavra.lower() in coluna.lower() for palavra in PALAVRAS_CHAVE):
            # Soma
            soma_valores = df_final_[coluna].sum(skipna=True)

            # Exibição em formato brasileiro
            valor_formatado = f"{soma_valores:,.2f}".replace(",", "X").replace(".", ",").replace("X", ".")

            resultado += f"\n📌 Coluna: {coluna} (🔄 Convertida para Float e Substituição de '.' por ',')\n"
            resultado += f"   🔹 Linhas Preenchidas: {linhas_preenchidas}\n"
            resultado += f"   🔹 Total Somado: {soma_valores:,.2f}\n"
    else:
        resultado += f"\n📌 Coluna: {coluna}\n"
        resultado += f"   🔹 Linhas Preenchidas: {linhas_preenchidas}\n"

# Copiar para a área de transferência
pyperclip.copy(resultado)

# Exibir os resultados
print("\n✅ Resultado copiado para a área de transferência!")


✅ Resultado copiado para a área de transferência!
